In [2]:
import numpy as np
import pandas as pd
import os
import json
import requests

pd.set_option('display.max_columns',None)

In [3]:
flight_df = pd.read_feather('data/v2_clean_flight')

In [4]:
def lat_lon_api(city):
    url = 'http://api.positionstack.com/v1/forward'
    position_stack_key = os.environ['position_stack_key']
    params = {'query':city,
                'access_key':position_stack_key,
                'limit': 1}
    response = requests.get(url, params=params).json()
    return response


In [5]:
sample_weather_df = flight_df[:100].copy()

In [6]:
def lat_grabber(df):
    lat_list = []
    lon_list = []
    for city in df['origin_city_name']:
        response = lat_lon_api(city)
        # print(city)
        # print(response)
        if 'latitude' not in response['data'][0]:
            lat_list.append('None')
        else:
            lat = response['data'][0]['latitude']
            lat_list.append(lat)
        if 'longitude' not in response['data'][0]:
            lon_list.append('None')
        else:
            lon = response['data'][0]['longitude']
            lon_list.append(lon)
    return lat_list,lon_list

In [7]:
lat_list, lon_list = lat_grabber(sample_weather_df)

In [ ]:
sample_weather_df['latitude'] = lat_list
sample_weather_df['longitude'] = lon_list
sample_weather_df

In [9]:
def weather_api(lat,lon,start):
    url = 'https://archive-api.open-meteo.com/v1/archive'
    params = {'latitude':lat,
                'longitude':lon,
                'start_date':start,
                'end_date':start,
                'hourly':'cloudcover',
                'daily':['rain_sum','snowfall_sum'],
                'timezone': 'America/New_York'
                }
    response = requests.get(url, params=params).json()
    return response

In [10]:
def weather_grabber(data):
    cloud_list = []
    rain_list = []
    snow_list = []
    sunny_list = []
    for info in data.values:
        date = info[0]
        lat = info[-2]
        lon = info[-1]
        response = weather_api(lat,lon,date)
        cloud = np.mean(response['hourly']['cloudcover'])
        sunny = 100 - cloud
        cloud_list.append(cloud)
        sunny_list.append(sunny)
        rain_list.append(response['daily']['rain_sum'][0])
        snow_list.append(response['daily']['snowfall_sum'][0])
    return cloud_list,rain_list,snow_list,sunny_list

In [11]:
cloud,rain,snow,sunny = weather_grabber(sample_weather_df)

In [ ]:
sample_weather_df['cloud(%)'] = cloud
sample_weather_df['sunny(%)'] = sunny
sample_weather_df['rain'] = rain
sample_weather_df['snow'] = snow
sample_weather_df

In [14]:
sample_weather_df.to_feather('data/flight_weather_100')